In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:


# Define the file name
csv_filename = f'simulation_results_eil51_8000_0.999_1000_cooling_types.csv'

# Read in the CSV file
df = pd.read_csv(csv_filename)

# Convert 'Distances' and 'Temperatures' from string to list (manual parsing)
df['Distances'] = df['Distances'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Temperatures'] = df['Temperatures'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Max Iterations'] = df['Max Iterations'].str.strip("[]").str.split(", ").apply(lambda x: list(map(int, x)))

# Plotting Distance vs Temperature (with temperature decreasing)
plt.figure(figsize=(12, 6))

cooling_types = df['Cooling Type'].unique()
for cooling_type in cooling_types:
    subset = df[df['Cooling Type'] == cooling_type]
    avg_distances = pd.DataFrame(subset['Distances'].tolist()).mean(axis=0)  # Average distances across runs
    avg_temperatures = pd.DataFrame(subset['Temperatures'].tolist()).mean(axis=0)  # Average temperatures across runs
    avg_iterations = subset['Max Iterations'].apply(lambda x: x[-1]).mean()  # Extract the last value and compute the average
    plt.plot(avg_temperatures, avg_distances, label=f'{cooling_type} Cooling (max iterations: {int(avg_iterations)})')

plt.title('Distance vs Temperature for Different Cooling Types')
plt.xlabel('Temperature (Decreasing)')
plt.ylabel('Distance')
plt.xscale('log')  # Log scale for better visualization
plt.gca().invert_xaxis()  # Reverse the x-axis to show temperature decreasing
plt.legend()
plt.grid(True)
plt.show()

# Plotting Distance Over Iterations
plt.figure(figsize=(12, 6))

# Plot for each cooling type
cooling_types = df['Cooling Type'].unique()
for cooling_type in cooling_types:
    subset = df[df['Cooling Type'] == cooling_type]
    avg_distances = pd.DataFrame(subset['Distances'].tolist()).mean(axis=0)  # Average distances across runs
    plt.plot(avg_distances, label=f'{cooling_type} Cooling')

plt.title('Distance Over Iterations for Different Cooling Types')
plt.xlabel('Iterations')
plt.ylabel('Distance')
plt.legend()
plt.grid(True)
plt.show()

# Convergence Plot: Stepwise Change in Distance
plt.figure(figsize=(12, 6))

for cooling_type in cooling_types:
    subset = df[df['Cooling Type'] == cooling_type]
    avg_distances = pd.DataFrame(subset['Distances'].tolist()).mean(axis=0)  # Average distances across runs
    stepwise_changes = abs(avg_distances.diff().fillna(0))  # Calculate stepwise change (distance[i] - distance[i-1])
    plt.plot(stepwise_changes, label=f'{cooling_type} Cooling')

plt.title('Stepwise Convergence: Distance Changes Over Iterations')
plt.xlabel('Iterations')
plt.ylabel('Change in Distance (ΔDistance)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting Distance vs Temperature (with temperature decreasing)
plt.figure(figsize=(12, 6))

for cooling_type in cooling_types:
    subset = df[df['Cooling Type'] == cooling_type]
    avg_distances = pd.DataFrame(subset['Distances'].tolist()).mean(axis=0)  # Average distances across runs
    avg_temperatures = pd.DataFrame(subset['Temperatures'].tolist()).mean(axis=0)  # Average temperatures across runs
    plt.plot(avg_temperatures, avg_distances, label=f'{cooling_type} Cooling')

plt.title('Distance vs Temperature for Different Cooling Types')
plt.xlabel('Temperature (Decreasing)')
plt.ylabel('Distance')
plt.xscale('log')  # Log scale for better visualization
plt.gca().invert_xaxis()  # Reverse the x-axis to show temperature decreasing
plt.legend()
plt.grid(True)
plt.show()

# Convergence Analysis
plt.figure(figsize=(12, 6))

# Plot convergence for each cooling type
for cooling_type in cooling_types:
    subset = df[df['Cooling Type'] == cooling_type]
    final_distances = subset['Distances'].apply(lambda x: x[-1])  # Get the final distance for each run
    plt.hist(final_distances, bins=15, alpha=0.7, label=f'{cooling_type} Cooling')

plt.title('Convergence Analysis: Final Distances Distribution')
plt.xlabel('Final Distance')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()
